# CSILSP-NCS-CS-FAL

In [1]:
import gurobipy as gb
from CSILSPGenerator import GeneratorTW_CS

#### Decision variables:

$$
Y_t = \begin{cases}
1 \text{ if } Z_{tk} > 0\\
0 \text{ otherwise }
\end{cases}
$$

$$
Z_{kt} = \begin{cases}
\text{ quantity produced/supplied
in period k } \\ \text{ to satisfy the demand in period t }
\end{cases}
$$

#### Parameters:

$$
T = \begin{cases}
\text{ number of periods in the horizon }
\end{cases}
$$

$$
d_t = \begin{cases}
\text{ Customer aggregate demand that must } \\ \text{ be delivered at the end of period t }
\end{cases}
$$

$$
h_t = \begin{cases}
\text{ Holding cost per unit of product in stock at the end of period t }
\end{cases}
$$

$$
p_t = \begin{cases}
\text{ Unit production/supplying cost in period t }
\end{cases}
$$

$$
s_t = \begin{cases}
\text{ Fixed setup cost in period t }
\end{cases}
$$

$$
C_t = \begin{cases}
\text{ Production/Supply capacity at period t }
\end{cases}
$$

#### CSILSP-NCS / CSILSP-CS (FAL)

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T}\sum_{k=t}^{T} (p_t + \sum_{j=t}^{k-1}h_j)Z_{tk} + \sum_{t=1}^{T}s_tY_t &\\
\text{s.t.} \;\;\;\;\;&\\
\sum_{k=1}^{t}Z_{kt} &= d_t \;\; , & \forall t \;\;\\
Z_{tk} &\leq d_kY_t \;\; , & \forall t,k \geq t \;\;\\
\sum_{k=t}^{T}Z_{tk} &\leq C_t \;\; , & \forall t \;\;\\
Z_{tk} &\geq 0 \;\; , & \forall t,k \geq t \;\;\\
\sum_{k=1}^{t_1}Z_{k,t_2} &\leq \sum_{k=1}^{t_1}d_{k,t_2} \;\; , & \forall t_1,t_2 (t_1 \leq t_2)\;\;\\
Y_t &\in \{0,1\} \;\; , & \forall t \;\;
\end{alignat}
$$

In [2]:
# generation parameters

if __name__ == '__main__':
    T = 8
    l_b = 10
    u_b = 20

In [3]:
# generate instance

if __name__ == '__main__' or __name__ == '__autoexec__':
    instance = GeneratorTW_CS(T)
    instance.generate(a=l_b,b=u_b,dw_a=l_b,dw_b=u_b)

In [4]:
if __name__ == '__main__':
    print(instance.printData())

       d     C   h   p   s
T                         
0    0.0  89.0  14  16  13
1   12.0  92.0  14  20  18
2   27.0  85.0  20  18  15
3   33.0  91.0  20  19  17
4   54.0  68.0  11  20  16
5   78.0  70.0  16  15  15
6   96.0  74.0  15  13  16
7  110.0  13.0  11  10  12


In [5]:
if __name__ == '__main__':
    print(instance.get_dw())

{(0, 0): 0, (0, 1): 12, (0, 2): 11, (0, 3): 10, (0, 4): 19, (0, 5): 16, (0, 6): 14, (0, 7): 20, (1, 1): 0, (1, 2): 16, (1, 3): 11, (1, 4): 11, (1, 5): 11, (1, 6): 14, (1, 7): 12, (2, 2): 0, (2, 3): 12, (2, 4): 13, (2, 5): 12, (2, 6): 15, (2, 7): 14, (3, 3): 0, (3, 4): 11, (3, 5): 20, (3, 6): 15, (3, 7): 19, (4, 4): 0, (4, 5): 19, (4, 6): 18, (4, 7): 20, (5, 5): 0, (5, 6): 20, (5, 7): 13, (6, 6): 0, (6, 7): 12, (7, 7): 0}


In [6]:
# variable bindings

T = instance.get_T()
d_w = instance.get_dw()
d = instance.get_d()
C = instance.get_C()
h = instance.get_h()
p = instance.get_p()
s = instance.get_s()

In [7]:
# model

csilsp_ncs_cs_fal = gb.Model()

Academic license - for non-commercial use only


In [8]:
# decision variables

Y = csilsp_ncs_cs_fal.addVars(T,vtype=gb.GRB.BINARY,name='Y')

Z = csilsp_ncs_cs_fal.addVars(T,T,vtype=gb.GRB.CONTINUOUS,name='Z')

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T}\sum_{k=t}^{T} (p_t + \sum_{j=t}^{k-1}h_j)Z_{tk} + \sum_{t=1}^{T}s_tY_t &
\end{alignat}
$$

In [9]:
# objective function

objexpr = gb.quicksum( ( p[t] + (gb.quicksum( h[j] for j in range(t,k) )) )*Z[t,k] for t in range(T) for k in range(t,T) ) \
          + Y.prod(s)

csilsp_ncs_cs_fal.setObjective(objexpr,gb.GRB.MINIMIZE)

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

$$
\begin{alignat}{3}
\sum_{k=1}^{t}Z_{kt} &= d_t \;\; , & \forall t \;\;
\end{alignat}
$$

In [10]:
# constraints

csilsp_ncs_cs_fal.addConstrs(( (gb.quicksum(Z[k,t] for k in range(t+1))) == d[t] for t in range(T) ),name='constrZ')

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

$$
\begin{alignat}{3}
Z_{tk} &\leq d_kY_t \;\; , & \forall t,k \geq t \;\;
\end{alignat}
$$

In [11]:
csilsp_ncs_cs_fal.addConstrs(( Z[t,k] <= d[k]*Y[t] for t in range(T) for k in range(t,T) ),name='constrZ2')

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

$$
\begin{alignat}{3}
\sum_{k=t}^{T}Z_{tk} &\leq C_t \;\; , & \forall t \;\;
\end{alignat}
$$

In [12]:
csilsp_ncs_cs_fal.addConstrs(( (gb.quicksum(Z[t,k] for k in range(t,T))) <= C[t] for t in range(T) ),name='constrZ3')

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

$$
\begin{alignat}{3}
Z_{tk} &\geq 0 \;\; , & \forall t,k \geq t \;\;\\
\end{alignat}
$$

In [13]:
for t in range(T):
    for k in range(t,T):
        Z[t,k].lb = 0.0

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

$$
\begin{alignat}{3}
\sum_{k=1}^{t_1}Z_{k,t_2} &\leq \sum_{k=1}^{t_1}d_{k,t_2} \;\; , & \forall t_1,t_2 (t_1 \leq t_2)\;\;
\end{alignat}
$$

In [14]:
csilsp_ncs_cs_fal.addConstrs(( (gb.quicksum(Z[k,t2] for k in range(t1+1))) <=\
                              (gb.quicksum(d_w[k,t2] for k in range(t1+1))) \
                              for t2 in range(T) for t1 in range(t2+1) ),name='constrCS_NCS_FAL')

csilsp_ncs_cs_fal.update()
csilsp_ncs_cs_fal.write('csilsp_ncs_cs_fal.lp')

In [15]:
# solve model

csilsp_ncs_cs_fal.optimize()

Optimize a model with 88 rows, 72 columns and 263 nonzeros
Variable types: 64 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 33 rows and 31 columns
Presolve time: 0.00s
Presolved: 55 rows, 41 columns, 186 nonzeros
Variable types: 33 continuous, 8 integer (8 binary)

Root relaxation: objective 1.517200e+04, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    15172.000000 15172.0000  0.00%     -    0s

Explored 0 nodes (17 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 15172 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.517200000000e+04, best bound 1.517200000000e+04, gap 0.0000%


In [16]:
if __name__ == '__main__':
    print("Soluzione ottima:",csilsp_ncs_cs_fal.ObjVal)

Soluzione ottima: 15172.0


In [17]:
Y

{0: <gurobi.Var Y[0] (value -0.0)>,
 1: <gurobi.Var Y[1] (value 1.0)>,
 2: <gurobi.Var Y[2] (value 1.0)>,
 3: <gurobi.Var Y[3] (value 1.0)>,
 4: <gurobi.Var Y[4] (value 1.0)>,
 5: <gurobi.Var Y[5] (value 1.0)>,
 6: <gurobi.Var Y[6] (value 1.0)>,
 7: <gurobi.Var Y[7] (value 1.0)>}

In [18]:
Z

{(0, 0): <gurobi.Var Z[0,0] (value 0.0)>,
 (0, 1): <gurobi.Var Z[0,1] (value 0.0)>,
 (0, 2): <gurobi.Var Z[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var Z[0,3] (value 0.0)>,
 (0, 4): <gurobi.Var Z[0,4] (value 0.0)>,
 (0, 5): <gurobi.Var Z[0,5] (value 0.0)>,
 (0, 6): <gurobi.Var Z[0,6] (value 0.0)>,
 (0, 7): <gurobi.Var Z[0,7] (value 0.0)>,
 (1, 0): <gurobi.Var Z[1,0] (value 0.0)>,
 (1, 1): <gurobi.Var Z[1,1] (value 12.0)>,
 (1, 2): <gurobi.Var Z[1,2] (value 0.0)>,
 (1, 3): <gurobi.Var Z[1,3] (value 0.0)>,
 (1, 4): <gurobi.Var Z[1,4] (value 0.0)>,
 (1, 5): <gurobi.Var Z[1,5] (value 0.0)>,
 (1, 6): <gurobi.Var Z[1,6] (value 0.0)>,
 (1, 7): <gurobi.Var Z[1,7] (value 0.0)>,
 (2, 0): <gurobi.Var Z[2,0] (value 0.0)>,
 (2, 1): <gurobi.Var Z[2,1] (value 0.0)>,
 (2, 2): <gurobi.Var Z[2,2] (value 27.0)>,
 (2, 3): <gurobi.Var Z[2,3] (value 33.0)>,
 (2, 4): <gurobi.Var Z[2,4] (value 22.0)>,
 (2, 5): <gurobi.Var Z[2,5] (value 0.0)>,
 (2, 6): <gurobi.Var Z[2,6] (value 0.0)>,
 (2, 7): <gurobi.Var Z[2,7] (v

In [19]:
# execution time
if __name__ == '__main__':
    print("Optimize time: %.5g sec" % csilsp_ncs_cs_fal.Runtime)

Optimize time: 0.047255 sec


In [20]:
# Write results to file

results = "----------------------------------------------------------------" + "\n"
results += " Parameter T = %d " % (T) + "\n"
results += " Parameters d,h,p,s,C random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Time window demands (d_w) random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Execution time = %.5g sec " % csilsp_ncs_cs_fal.Runtime + "\n"
results += "----------------------------------------------------------------" + "\n\n\n"

In [21]:
with open("csilsp_ncs_cs_fal_results.txt", "a") as write_file:
    write_file.write(results)